In [1]:
from data_loader import *
import csv
from pycocotools.coco import COCO
from tqdm import tqdm
from vocab import *
import nltk
import os
import torchvision.transforms as transforms

In [13]:
batch_size = 2
vocab = Vocab()
nltk.download('punkt')

Successfully loaded vocabulary from wordDict.pkl and wordStat.pkl


[nltk_data] Downloading package punkt to /home/mtranquo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
annos = './data/annotations/captions_train2014.json'
images = '/datasets/COCO-2015/train2014/'
coco = COCO(annos)
trainIds = []

# get ids of training images
with open('TrainImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    trainIds = list(reader)
trainImgIds = [int(i) for i in trainIds[0]]

for imgId in trainImgIds:
    for caption in coco.imgToAnns[imgId]:
        trainIds.append(caption['id'])

train_loader = get_loader(root=images, json=annos, ids=trainIds, vocab=vocab,
                         transform=transforms.Resize((256,256)), batch_size=batch_size, shuffle=True, num_workers=4)

loading annotations into memory...
Done (t=0.82s)
creating index...
index created!
loading annotations into memory...
Done (t=0.76s)
creating index...
index created!


In [15]:
annos = './data/annotations/captions_val2014.json'
images = '/datasets/COCO-2015/val2014/'
coco = COCO(annos)
testIds = []

# get ids of test
with open('TestImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    testIds = list(reader)
testImgIds = [int(i) for i in testIds[0]]

for imgId in testImgIds:
    for caption in coco.imgToAnns[imgId]:
        testIds.append(caption['id'])

test_loader = get_loader(root=images, json=annos, ids=testIds, vocab=vocab,
                         transform=transforms.Resize((256,256)), batch_size=batch_size, shuffle=True, num_workers=4)

loading annotations into memory...
Done (t=0.42s)
creating index...
index created!
loading annotations into memory...
Done (t=0.50s)
creating index...
index created!


In [17]:
for iter, (images, targets, lengths) in enumerate(train_loader):
    print(targets.shape)
    break

torch.Size([2, 17])


In [9]:
# find the min width and height to crop
annos = './data/annotations/captions_train2014.json'
root = '/datasets/COCO-2015/train2014/'
coco = COCO(annos)
minWidth = 1000000
minHeight = 1000000
for imgId in trainImgIds:
    path = coco.loadImgs(imgId)[0]['file_name']
    image = Image.open(os.path.join(root, path)).convert('RGB')
    width, height = image.size
    minWidth = min(minWidth, width)
    minHeight = min(minHeight, height)

loading annotations into memory...
Done (t=0.99s)
creating index...
index created!


In [10]:
print(minWidth, minHeight)

120 104
